# Assignment 2

In [1]:
!pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.0 MB/s eta 0:00:00


## **Loading the dataset**

In [2]:
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/humor/val.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/train.csv
!wget https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/sarcasm/val.csv

--2025-04-04 16:41:38--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 406615 (397K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 397.08K  2.03MB/s    in 0.2s    

2025-04-04 16:41:39 (2.03 MB/s) - ‘train.csv’ saved [406615/406615]

--2025-04-04 16:41:39--  https://raw.githubusercontent.com/islnlp/Assignment_1_2025/refs/heads/main/hate/val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49821 (49K)

In [80]:
!wget https://github.com/rohitptnk/Machine-Learning-Projects/raw/refs/heads/main/NLP%20Assignments/Assignment%202/best_model_ffnn.keras
!wget https://github.com/rohitptnk/Machine-Learning-Projects/raw/refs/heads/main/NLP%20Assignments/Assignment%202/best_model_ffnn1.keras
!wget https://github.com/rohitptnk/Machine-Learning-Projects/raw/refs/heads/main/NLP%20Assignments/Assignment%202/best_model_ffnn2.keras

--2025-04-04 18:02:02--  https://github.com/rohitptnk/Machine-Learning-Projects/raw/refs/heads/main/NLP%20Assignments/Assignment%202/best_model_ffnn.keras
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rohitptnk/Machine-Learning-Projects/refs/heads/main/NLP%20Assignments/Assignment%202/best_model_ffnn.keras [following]
--2025-04-04 18:02:03--  https://raw.githubusercontent.com/rohitptnk/Machine-Learning-Projects/refs/heads/main/NLP%20Assignments/Assignment%202/best_model_ffnn.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8557331 (8.2M) [application/octet-stream]
Saving to: ‘best_model

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, f1_score


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Embedding,
    Dense,
    Flatten,
    Bidirectional,
    SimpleRNN,
    Input,
    Concatenate,
    Dropout
)
from tensorflow.keras.optimizers import Adam, AdamW, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryFocalCrossentropy, CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras_tuner as kt
from keras_tuner import RandomSearch, Objective

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [44]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/val.csv')

texts = train_data['Sentence'].values
labels = train_data['Tag'].values

train_data.sample(5)

,Sentence,Tag
3281,"Kiran kher ji ki khud ki beti Ka rape hota to bhi yahi bayan deti vo! Samjhe! BJP Raj me hua rape bhi deshhit me aur Rashtrawadi rape hota Hai Hai yar, samjhte nahi tum. #KyunkiSharmInheAatiNahi",1
3150,Yeh sab hindu satan mei hota hy. Bus mei rape. Rickshaw mei rape. Baap beti ko bech k paisay kamata hy. Larki Ki padaish py usko zameen mei,1
1595,baise hi hai kai si aurto ka rape krte waqt jai sri raam bolna aur jo allah hu akbar bol ke jaan leta hai bo islam nahi hai,0
2583,isi waja se aaj itna hate mila mujhe,0
2710,Sab se pehle qanoon vevastha darustha kijye kiy aapne bajrangyon gundagardi aur juned ka murder ni dekha?,0


In [45]:
'''!git clone https://github.com/rohitptnk/Machine-Learning-Projects.git
%cd Machine-Learning-Projects/NLP\ Assignments/Assignment\ 2'''

'!git clone https://github.com/rohitptnk/Machine-Learning-Projects.git\n%cd Machine-Learning-Projects/NLP\\ Assignments/Assignment\\ 2'

## **Pre-processing**

In [46]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(tokens)

texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [47]:
sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

embedding_dim = 100
window_size = 5

'''def generate_skipgram_pairs(sentences, window_size, vocab_size):
    skipgrams = []

    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word not in tokenizer.word_index:
                continue

            target_word = tokenizer.word_index[word]

            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]

            for context_word in context_window:
                if context_word != word and context_word in tokenizer.word_index:
                    context_word_idx = tokenizer.word_index[context_word]
                    skipgrams.append([target_word, context_word_idx])

    return np.array(skipgrams)


skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)


X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

# Embedding layer for target and context
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

# cosine similarity between target and context
dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')

labels = np.ones((len(skipgrams), 1))
word2vec_model.fit([X_target, X_context], labels, epochs=10, batch_size=128)
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]'''



'def generate_skipgram_pairs(sentences, window_size, vocab_size):\n    skipgrams = []\n\n    for sentence in sentences:\n        for i, word in enumerate(sentence):\n            if word not in tokenizer.word_index:\n                continue\n\n            target_word = tokenizer.word_index[word]\n\n            context_window = sentence[max(i - window_size, 0): min(i + window_size + 1, len(sentence))]\n\n            for context_word in context_window:\n                if context_word != word and context_word in tokenizer.word_index:\n                    context_word_idx = tokenizer.word_index[context_word]\n                    skipgrams.append([target_word, context_word_idx])\n\n    return np.array(skipgrams)\n\n\nskipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)\n\n\nX_target, X_context = zip(*skipgrams)\nX_target = np.array(X_target)\nX_context = np.array(X_context)\n\ninput_target = tf.keras.layers.Input(shape=(1,))\ninput_context = tf.keras.layers.Input(shape=(

In [48]:
# np.save('word2vec_embeddings.npy', trained_embeddings)

In [49]:
# trained_embeddings = np.load('word2vec_embeddings.npy')

In [50]:
'''X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = train_data['Tag'].values

X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)'''


"X = pad_sequences(sequences, maxlen=max_sequence_len)\nlabels = train_data['Tag'].values\n\nX_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)"

In [51]:
test_texts = test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

In [52]:
# custom macro F1 score function
def macro_f1_score(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_pred_bin = tf.round(y_pred)

    def f1(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())

        f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_val

    f1_per_class = f1(y_true, y_pred_bin)
    return K.mean(f1_per_class)

In [53]:
# Function to build the FFNN model for tuning
'''def build_model(hp):
    model = Sequential()

    embedding_dim = 100

    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)

    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)

    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))  # Assuming 5 classes

    optimizer = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)
    model.compile(optimizer=opt, loss=BinaryFocalCrossentropy(), metrics=[macro_f1_score])

    return model

objective = Objective('val_macro_f1_score', direction='max')

# Hyperparameter search
tuner = RandomSearch(
    build_model,
    objective=objective,
    max_trials=15,
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='ffnn_tuning_v7.2213123123'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

best_model_ffnn = tuner.get_best_models(num_models=1)[0]
best_model_ffnn.summary()'''

"def build_model(hp):\n    model = Sequential()\n\n    embedding_dim = 100\n\n    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)\n\n    embedding_layer = Embedding(\n        input_dim=vocab_size,\n        output_dim=embedding_dim,\n        input_length=sequence_len,\n        weights=[trained_embeddings],\n        trainable=False\n    )\n\n    model.add(embedding_layer)\n\n    model.add(Flatten())\n\n    num_layers = hp.Int('num_layers', min_value=1, max_value=4)\n\n    for i in range(num_layers):\n        model.add(Dense(\n            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),\n            activation='tanh'\n        ))\n\n    # Output layer\n    model.add(Dense(1, activation='sigmoid'))  # Assuming 5 classes\n\n    optimizer = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])\n\n    if optimizer == 'Adam':\n        opt = Adam(learning_rate=0.001)\n    elif optimizer == 'AdamW':\n        opt = AdamW(learning_rate=0.001)\n    else

## **Saving the models**

In [54]:
# best_model_ffnn.save('best_model_ffnn.keras')

## **Loading the saved models**

In [55]:
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn.keras', custom_objects={'macro_f1_score': macro_f1_score})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [56]:
def print_macro_f1(classification_report_dict, model_name):
    macro_f1 = classification_report_dict['macro avg']['f1-score']
    print(f"{model_name} Model Macro Average F1-score: {macro_f1:.4f}")

## **Testing and Evaluation**

In [57]:
test_labels = test_data['Tag'].values

models = {
    "FFNN": best_model_ffnn,
}

reports = {}

for model_name, model in models.items():
    predictions = model.predict(X_test)
    predictions = (predictions > 0.5).astype(int)

    report = classification_report(
        test_labels,
        predictions,
        target_names=[0,1],
        output_dict=True
    )


    reports[model_name] = report

    print_macro_f1(report, model_name)

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
FFNN Model Macro Average F1-score: 0.4726


# Humour

In [58]:
train_data = pd.read_csv('/content/train.csv.1')
test_data = pd.read_csv('/content/val.csv.1')

texts = train_data['Sentence'].values
labels = train_data['Tag'].values

train_data.sample(15)

,Sentence,Tag
1567,"Misbah-ul-haq wants to make love.\nSings, ""Misbahon ke darmiyan, do pyar mil rahe hain...""",1
1724,.@ashutosh83B bhai taiyaar ho jaa.. aaj tujhe jalaenge,1
1519,"Koi baat nahi, Pakistan ki to maari thi na hum logo ne. Chill.",0
956,"Jamai Raja, Kumkum Bhagya, Diya Aur Baati Hum etc etc. \n#ThingsThatUniteIndians",1
912,Raina usko mazza Chatara hai #IndvsZim,0
1977,Ram Kapoor: Meri biwi mujhe bol rahi hai ki meri jaan le lo.\n\nAll married man: BHENCHO WHAT ARE YOU WAITING FOR?,1
941,Jee karda.. marjaniya.. #badlapur #OnLoop,0
581,5 - 5 karke tamatar phenkopic.twitter.com/a55ucfBgZk,1
634,Hahahahaha ab to gaali dene ka mann bhi nahi kar raha. Hasi aa rahi hai chutiyo pe,1
276,Dhoni better make up for his decision making with his batting. Nahi to ghar ghar mein aaj Yograj Singh jaag uthega #IndvsPak,1


## **Pre-processing**

In [59]:
texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [60]:

sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

embedding_dim = 100
window_size = 5

'''skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)

X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')

labels = np.ones((len(skipgrams), 1))  # Positive samples are labeled 1

word2vec_model.fit([X_target, X_context], labels, epochs=10, batch_size=128)
trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]'''

'skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)\n\nX_target, X_context = zip(*skipgrams)\nX_target = np.array(X_target)\nX_context = np.array(X_context)\n\ninput_target = tf.keras.layers.Input(shape=(1,))\ninput_context = tf.keras.layers.Input(shape=(1,))\n\nembedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")\ntarget_embedding = embedding(input_target)\ncontext_embedding = embedding(input_context)\n\ntarget_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)\ncontext_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)\n\ndot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])\noutput = tf.keras.layers.Dense(1, activation=\'sigmoid\')(dot_product)\n\nword2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)\nword2vec_model.compile(optimizer=\'adam\', loss=\'binary_crossentropy\')\n\nlabels = np.ones((le

In [61]:
# np.save('word2vec_embeddings1.npy', trained_embeddings)

In [62]:
# trained_embeddings = np.load('word2vec_embeddings1.npy')

In [63]:
'''X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = train_data['Tag'].values

X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)
'''

"X = pad_sequences(sequences, maxlen=max_sequence_len)\nlabels = train_data['Tag'].values\n\nX_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)\n"

In [64]:
test_texts = test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

## **Training**

In [65]:
'''def build_model(hp):
    model = Sequential()

    embedding_dim = 100

    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)

    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())

    num_layers = hp.Int('num_layers', min_value=1, max_value=4)

    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(optimizer=opt, loss=BinaryFocalCrossentropy(), metrics=[macro_f1_score])

    return model

objective = Objective('val_macro_f1_score', direction='max')

tuner = RandomSearch(
    build_model,
    objective=objective,
    max_trials=15,
    executions_per_trial=1,
    directory='hyperparam_tuning2',
    project_name='ffnn_tuning_v7.2213123123'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
best_model_ffnn = tuner.get_best_models(num_models=1)[0]
best_model_ffnn.summary()'''

"def build_model(hp):\n    model = Sequential()\n\n    embedding_dim = 100\n\n    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)\n\n    embedding_layer = Embedding(\n        input_dim=vocab_size,\n        output_dim=embedding_dim,\n        input_length=sequence_len,\n        weights=[trained_embeddings],\n        trainable=False\n    )\n\n    model.add(embedding_layer)\n    model.add(Flatten())\n\n    num_layers = hp.Int('num_layers', min_value=1, max_value=4)\n\n    for i in range(num_layers):\n        model.add(Dense(\n            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),\n            activation='tanh'\n        ))\n\n    model.add(Dense(1, activation='sigmoid'))\n\n    optimizer = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])\n\n    if optimizer == 'Adam':\n        opt = Adam(learning_rate=0.001)\n    elif optimizer == 'AdamW':\n        opt = AdamW(learning_rate=0.001)\n    else:\n        opt = SGD(learning_rate=0.001)\n\

In [66]:
# best_model_ffnn.save('best_model_ffnn1.keras')

In [67]:
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn1.keras', custom_objects={'macro_f1_score': macro_f1_score})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [68]:
test_labels = test_data['Tag'].values

models = {
    "FFNN": best_model_ffnn,
}

reports = {}

for model_name, model in models.items():
    predictions = model.predict(X_test)
    predictions = (predictions > 0.5).astype(int)

    report = classification_report(
        test_labels,
        predictions,
        target_names=[0,1],
        output_dict=True
    )

    reports[model_name] = report

    print_macro_f1(report, model_name)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
FFNN Model Macro Average F1-score: 0.6228


# **Sarcasm**

In [69]:

train_data = pd.read_csv('/content/train.csv.2')
test_data = pd.read_csv('/content/val.csv.2')

texts = train_data['Sentence'].values
labels = train_data['Tag'].values

train_data.sample(15)

,Sentence,Tag
3771,Manoj ji ye aam aadmi hai logo ke liye apni politics to banati hai,0
781,"4 din me 2 accidents, kuch to jhol hai,shayad politics ho rahi hai..",0
3065,Ghotala kumar police ne arrest Kiya vo Nahi dekha?? Yahi mullavadi aur parivarvadi politics gart me gira rahi hai tum ko aur congress ko,0
2471,Chutiyo ke chacha jan politics hi kahe nahi join kar lete Ashutoswa ki tarah kabhi saram ati hai ka,0
2020,Akhir tum jaise qoum ke bewakoof kuch nahi karenge toh moulana ko yeh sab karna hi padega na politics hamare liye masla hai solve karna hai,0
2162,@ShazaFK agr ap waiqi insaniyt k liye #KPK gae ho to..... phir elaaj he kro.... ye elaaj k naam pe politics q kr rahay ho????,0
2734,Nitish kumar ki gandi politics ab ek ek kr samne aayega,0
3800,Abhi tak Hame cricket me out karne wala Paida bhi nahi hua hai agar Hame agar koi cricket me out kar de usse 101 rupay Diya karate the pic.twitter.com/UJc2GtlfW8,0
4075,"370 aur 35A hat kar rahegi , triple talaq ki tarah , kuchh bhi kar lo.....",0
231,"Bhai @iamsrk ki Raees dekhne chalein? ""Bro paise nahi hain."" #irony #Raees #MonthEnd",1


### Pre-processing

In [70]:
texts_preprocessed = [preprocess_text(text) for text in texts]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_preprocessed)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts_preprocessed)

max_sequence_len = 128
X = pad_sequences(sequences, maxlen=max_sequence_len)

In [71]:

sentences = [text.split() for text in texts_preprocessed]
vocab_size = len(tokenizer.word_index) + 1

embedding_dim = 100
window_size = 5

'''skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)

X_target, X_context = zip(*skipgrams)
X_target = np.array(X_target)
X_context = np.array(X_context)

input_target = tf.keras.layers.Input(shape=(1,))
input_context = tf.keras.layers.Input(shape=(1,))

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

target_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)
context_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)

dot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])
output = tf.keras.layers.Dense(1, activation='sigmoid')(dot_product)

word2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
word2vec_model.compile(optimizer='adam', loss='binary_crossentropy')

labels = np.ones((len(skipgrams), 1))
word2vec_model.fit([X_target, X_context], labels, epochs=10, batch_size=128)

trained_embeddings = word2vec_model.get_layer('embedding').get_weights()[0]'''



'skipgrams = generate_skipgram_pairs(sentences, window_size, vocab_size)\n\nX_target, X_context = zip(*skipgrams)\nX_target = np.array(X_target)\nX_context = np.array(X_context)\n\ninput_target = tf.keras.layers.Input(shape=(1,))\ninput_context = tf.keras.layers.Input(shape=(1,))\n\nembedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name="embedding")\ntarget_embedding = embedding(input_target)\ncontext_embedding = embedding(input_context)\n\ntarget_embedding = tf.keras.layers.Reshape((embedding_dim,))(target_embedding)\ncontext_embedding = tf.keras.layers.Reshape((embedding_dim,))(context_embedding)\n\ndot_product = tf.keras.layers.Dot(axes=1)([target_embedding, context_embedding])\noutput = tf.keras.layers.Dense(1, activation=\'sigmoid\')(dot_product)\n\nword2vec_model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)\nword2vec_model.compile(optimizer=\'adam\', loss=\'binary_crossentropy\')\n\nlabels = np.ones((le

In [72]:
# np.save('word2vec_embeddings2.npy', trained_embeddings)

In [73]:
# trained_embeddings = np.load('word2vec_embeddings2.npy')

In [74]:
'''X = pad_sequences(sequences, maxlen=max_sequence_len)
labels = train_data['Tag'].values

X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)'''


"X = pad_sequences(sequences, maxlen=max_sequence_len)\nlabels = train_data['Tag'].values\n\nX_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.1, random_state=60)"

In [75]:
test_texts = test_data['Sentence'].values
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_preprocessed)
X_test = pad_sequences(test_sequences, maxlen=max_sequence_len)

### Training

In [76]:
'''def build_model(hp):
    model = Sequential()

    embedding_dim = 100
    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=sequence_len,
        weights=[trained_embeddings],
        trainable=False
    )

    model.add(embedding_layer)
    model.add(Flatten())
    num_layers = hp.Int('num_layers', min_value=1, max_value=4)

    for i in range(num_layers):
        model.add(Dense(
            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),
            activation='tanh'
        ))

    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])

    if optimizer == 'Adam':
        opt = Adam(learning_rate=0.001)
    elif optimizer == 'AdamW':
        opt = AdamW(learning_rate=0.001)
    else:
        opt = SGD(learning_rate=0.001)

    model.compile(optimizer=opt, loss=BinaryFocalCrossentropy(), metrics=[macro_f1_score])

    return model

objective = Objective('val_macro_f1_score', direction='max')

tuner = RandomSearch(
    build_model,
    objective=objective,
    max_trials=15,
    executions_per_trial=1,
    directory='hyperparam_tuning3',
    project_name='ffnn_tuning_v7.2213123123'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

best_model_ffnn = tuner.get_best_models(num_models=1)[0]

best_model_ffnn.summary()'''

"def build_model(hp):\n    model = Sequential()\n\n    embedding_dim = 100\n    sequence_len = hp.Int('sequence_len', min_value=64, max_value=128, step=16)\n    embedding_layer = Embedding(\n        input_dim=vocab_size,\n        output_dim=embedding_dim,\n        input_length=sequence_len,\n        weights=[trained_embeddings],\n        trainable=False\n    )\n\n    model.add(embedding_layer)\n    model.add(Flatten())\n    num_layers = hp.Int('num_layers', min_value=1, max_value=4)\n\n    for i in range(num_layers):\n        model.add(Dense(\n            units=hp.Int(f'dense_units_{i+1}', min_value=32, max_value=64, step=16),\n            activation='tanh'\n        ))\n\n    model.add(Dense(1, activation='sigmoid'))\n\n    optimizer = hp.Choice('optimizer', ['Adam', 'AdamW', 'SGD'])\n\n    if optimizer == 'Adam':\n        opt = Adam(learning_rate=0.001)\n    elif optimizer == 'AdamW':\n        opt = AdamW(learning_rate=0.001)\n    else:\n        opt = SGD(learning_rate=0.001)\n\n    m

In [77]:
# best_model_ffnn.save('best_model_ffnn2.keras')

In [78]:
best_model_ffnn = tf.keras.models.load_model('best_model_ffnn2.keras', custom_objects={'macro_f1_score': macro_f1_score})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [79]:

test_labels = test_data['Tag'].values

models = {
    "FFNN": best_model_ffnn,
}

reports = {}

for model_name, model in models.items():
    predictions = model.predict(X_test)
    predictions = (predictions > 0.5).astype(int)

    report = classification_report(
        test_labels,
        predictions,
        target_names=[0,1],
        output_dict=True
    )

    reports[model_name] = report

    print_macro_f1(report, model_name)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
FFNN Model Macro Average F1-score: 0.7630
